<a href="https://colab.research.google.com/github/kriegsmar/MINE_VA/blob/master/MINE_4201_SR_Laboratorio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MINE 4201- SR -  Laboratorio 1 
# Exploración modelos KNN

Henry Fabra, César Gómez, William Morales


Vamos a utilizar las siguientes librerias en el laboratorio


In [ ]:
!pip install numpy
!pip install scikit-surprise
!pip install pandas
!pip install seaborn
!pip install surprise

In [ ]:
pwd

'/content'

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import KNNBasic
from surprise import accuracy
import random

#Para garantizar reproducibilidad en resultados
seed = 10
random.seed(seed)
np.random.seed(seed)


## Suba al servidor los archivos u.data y u.item que se encuentran en el dataset descargado, en la pestaña files

In [ ]:
if not (os.path.exists('content/u.data') and os.path.exists('content/u.item')):
  raise ValueError('Los archivos u.data e u.item no fueron encontrados en el path')
else:
  print("Los archivos han sido cargados")

ValueError: ignored

In [ ]:
ratings=pd.read_csv('content/u.data', sep = '\t', names = [ 'user_id', 'item_id', 'rating', 'timestamp' ] )

FileNotFoundError: ignored

In [ ]:
items=pd.read_csv('/content/u.item', sep = '\|', names = ['movie id' ,'movie title','release date','video release date','IMDb URL ','unknown',
                                                          'Action','Adventure','Animation','Children','Comedy','Crime','Documentary','Drama',
                                                          'Fantasy','Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western'], encoding='latin-1', engine='python' )

Verifique que los ratings y los items hayan sido cargados correctamente





In [ ]:
ratings.head()


In [ ]:
ratings

In [ ]:
# Se cambia el índice por la columna movie id
items=items.set_index(items['movie id'])

In [ ]:
items.head()

## Visualice la distribución de ratings, ¿Qué puede decir al respecto?

In [ ]:
sns.countplot(y='rating', data=ratings)

In [ ]:
sns.displot(y='rating', data=ratings, kde=True)

## En la siguiente celda se esta calculando el número de ratings por usuario y  el promedio de sus calificaciones, el dataframe esta ordenado por el número de calificaciones

In [ ]:
ratings.groupby('user_id')['rating'].agg({'count','mean'}).sort_values(by='count', ascending=False)

##Utilice las siguientes celdas para encontrar respuesta a las siguientes preguntas

¿Cómo es la distribución del número de ratings por usuario?




In [ ]:
rating_grouped = ratings.groupby('user_id')['rating'].agg({'count','mean'}).sort_values(by='count', ascending=False)

In [ ]:
rating_grouped.head()

In [ ]:
sns.displot(data=rating_grouped, x ="count")

In [ ]:
sns.displot(data=rating_grouped, x="user_id", y ="count")

¿Cómo es la distribución del promedio de calificación por usuario?

In [ ]:
sns.displot(data=rating_grouped, x="user_id", y="mean")

Al revisar cuales son las calificaciones de las películas se logra identificar que una gran variedad de las películas tiene clasificaciones entre 3 y 4, debido a que es en este rango donde se ve la mayor aglomeración de datos en los gráficos.

##En la siguiente celda se esta calculando el número de ratings por items, el promedio de sus calificaciones y la varianza de sus calificaciones. 

In [ ]:
ratings.groupby('item_id')['rating'].agg({'count','mean','var'}).join(items['movie title'], how='left')

In [ ]:
ratings.groupby('item_id')['rating'].agg({'count','mean','var','std'}).join(items['movie title'], how='left')

##Utilice las siguientes celdas para encontrar la respuesta a las siguientes preguntas

¿Cuáles son los items con más calificaciones?  





In [ ]:
calificacion_item= ratings.groupby('item_id')['rating'].agg({'count','mean','var'}).join(items['movie title'], how='left').sort_values(by='count', ascending=False)

In [ ]:
calificacion_item.head()

In [ ]:
calificacion_item.shape

In [ ]:
calificacion_item['item_id'] = calificacion_item.index

¿Se puede observar el fenomeno de cola larga en este dataset?, evidencielo por medio de un gráfico


In [ ]:
sns.barplot(x='item_id', y="count", data=calificacion_item, order=calificacion_item["item_id"])

In [ ]:
sns.displot(calificacion_item, x="count")

In [ ]:
sns.displot(ratings, x="item_id")

## Creación de listas de recomendación no personalizadas

Genere inicialmente una lista de recomendación de tamaño 10 no personalizada con los items con mejor promedio, en teoría estos serían los mejores items del dataset y una buena lista inicial con los elementos más aclamados por los usuarios del sistema

¿Qué problemas tiene generar una lista no personalizada solamente con el promedio?

¿Es posible generar una mejor lista de recomendación teniendo en cuenta que tanto el número de ratings del item como su promedio? 

Cree una lista top 10 con los mejores items globales siguiendo la estrategia seleccionada

Ver: 

http://www.evanmiller.org/how-not-to-sort-by-average-rating.html

https://es.wikipedia.org/wiki/Intervalo_de_confianza


In [ ]:
sr_noPersonalizada = calificacion_item.sort_values(by='mean',ascending=False)[0:10]

In [ ]:
sr_noPersonalizada.shape

In [ ]:
sr_noPersonalizada

Al crear una lista no personalizada solo con el promedio de la película se puede presentar el problema que no se considere aspectos como el número de veces que la película fue vista o el número de usuarios que la vieron, generando que se recomiende un item que solo fue visto una vez, como en el caso puntual de este sistema de recomendación, que se logra ver que fueron calificadas por máximo por 3 usuarios y tienen calificación perfecta.

Si se considera tener en cuental, el promedio de alificación de l apelícula y el número de veces que se ha visto, esta alternativa puede generar un mejor sistema de recomendación debido a que esta considerando la relación que existe entre estos dos aspectos, lo que permite que al usuario se le pueda mostrar una película que realmente pueda ser tenida en cuenta por el usuario.


In [ ]:
cd_intConfianza = ratings.groupby('item_id')['rating'].agg({'count','mean','var','std'}).join(items['movie title'], how='left')

In [ ]:
cd_intConfianza

In [ ]:
def limite_inferior_confianza(df):
  error_estandar = df['std']/df['count']
  limite_inferior =df['mean']-(1.96*error_estandar)
  return limite_inferior
lim_inferior = limite_inferior_confianza(cd_intConfianza)
cd_intConfianza['intervalo_confianza_lim_inferior'] = lim_inferior 

In [ ]:
def limite_superior_confianza(df):
  error_estandar = df['std']/df['count']
  limite_superior =df['mean']+(1.96*error_estandar)
  return limite_superior
lim_superior = limite_superior_confianza(cd_intConfianza)
cd_intConfianza['intervalo_confianza_lim_superior'] = lim_superior

In [ ]:
cd_intConfianza

In [ ]:
cd_intConfianza['delta_ci'] = cd_intConfianza['intervalo_confianza_lim_superior']- cd_intConfianza['mean']
sr_personalizado = cd_intConfianza[(cd_intConfianza['delta_ci']<=0.1) & (cd_intConfianza['mean']>=3.5)]
sr_personalizado.shape

Con la nueva lista personalizada se procede a realizar un filtro de las peliculas de acuerdo a la media de cada una y al diferencial entre la media del item y el limite de confianza superior.

In [ ]:
sr_personalizado_final = sr_personalizado.sort_values(by='mean', ascending=False)
sr_personalizado_final[:10]

Como se puede ver al usar la estrategia de tener en cuenta el intervalo de confianza para las mejores películas a recomendar, se obtienen resultados de items que han sido calificados varias veces por los usuarios. aunque sucede algo similar al caso anterior de que se van a recomendar películas que han sido vistas menos de 5 veces, también están los casos con otras que han sido vistos hasta 200 veces, lo que permite crear una lista de recomendación que tiene en cuenta aquellos titulos populares pero también titulos no tan conocidos y que pueden ser del gusto del usuario. 

#Creación del dataset de entrenamiento y prueba

El próximo paso es cargar los datos a surprise, para esto vamos a inicializar un modelo de datos de surprise con la información de los ratings cargados en el dataframe

In [ ]:
reader = Reader( rating_scale = ( 1, 5 ) )
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings[ [ 'user_id', 'item_id', 'rating' ] ], reader )

Se divide el dataset en entrenamiento y test. El dataset de entrenamiento será presentado al algoritmo de recomendación y el dataset de test nos servirá para medir qué tan buenas fueron las predicciones del sistema de recomendación. El conjunto de test generado es una lista con los ratings de prueba

In [ ]:
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)

In [ ]:
train_set.n_ratings

In [ ]:
len(test_set)

In [ ]:
#Este es el primer elemento del dataset de prueba (usuario 154, pelicula 302, rating 4)
test_set[0]

## Creación de modelo de filtrado colaborativo basado en similitud con usuarios o items cercanos

Surprise cuenta con la implementación de los modelos colaborativos dentro de la clase [KNNBasic](https://surprise.readthedocs.io/en/stable/knn_inspired.html) 

El modelo recibe los siguientes parámetros: 


*   k: El máximo número de vecinos con el que se hará la extrapolación
*   min_k : El mínimo número de vecinos con el que se extrapolará un rating
*   sim_options : Opciones de similitud pasadas como un diccionario de python, aqui se le configura al modelo el tipo de similitud a usar para encontrar los vecinos y si la extrapolación debe hacerse usando usuarios o items similares. Revise el formato y similitudes disponibles en surprise en [este link](https://surprise.readthedocs.io/en/stable/prediction_algorithms.html#similarity-measure-configuration)





In [ ]:
# se crea un modelo knnbasic item-item con similitud coseno 
sim_options = {'name': 'cosine',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=20, min_k=2, sim_options=sim_options)

In [ ]:
#Se le pasa la matriz de utilidad al algoritmo 
algo.fit(trainset=train_set)

Una vez cargados los ratings al modelo, se puede realizar una predicción para un usuario, en este caso vamos a calcular la predicción que el modelo esta realizando para la primera entrada del dataset de test la predicción para el usuario con id 154 y el item con id 302, que corresponde a la película L.A. Confidential (1997)

---



In [ ]:
#Verifique la propiedad est de la predicción
algo.predict(154,302)

In [ ]:
items[items['movie id']==302]

Como podemos ver, la predicción (4.24) del modelo no esta alejada de lo que realmente opinó el usuario  (4.0)

Para medir la calidad de la predicción para todos los usuarios e items del dataset de prueba, vamos a comparar lo que dice el modelo de predicción vs lo que dice el conjunto de prueba, para esto vamos a usar la métrica [RMSE](https://surprise.readthedocs.io/en/stable/accuracy.html#surprise.accuracy.rmse)

Inicialmente calculamos la predicción para todos los elementos del conjunto de test

In [ ]:
test_predictions=algo.test(test_set)

In [ ]:
#5 primeras predicciones
test_predictions[0:5]

Ahora se mide el RMSE de las predicciones vs el valor del dataset

In [ ]:
# En promedio, el sistema encuentra ratings que estan una estrella por encima o por debajo del rating del usuario
accuracy.rmse( test_predictions, verbose = True )

##Utilice las siguientes celdas para encontrar la respuesta a las siguientes preguntas

¿Cuál es el RMSE de un modelo usuario-usuario con los mismos parámetros de similitud?


In [ ]:
# se crea un modelo knnbasic item-item con similitud coseno 
sim_options = {'name': 'cosine',
               'user_based': True  # calcule similitud usuario-usuario
               }
usuario_usuario = KNNBasic(k=20, min_k=2, sim_options=sim_options)
usuario_usuario.fit(trainset=train_set)
test_predictions_user=usuario_usuario.test(test_set)
test_predictions_user[:10]

In [ ]:
accuracy.rmse( test_predictions_user, verbose = True )

Al revisar el valor del RMSE del modelo usuario-usuario comparandolo con el modelo entre items, el resultado es similar y muestra que el RMSE está alrededor de 1.

¿Cuál es el efecto de cambiar el número de vecinos en la calidad del modelo usuario-usuario ? 

Pruebe para un número diferente de vecinos (e.j 5, 10, 20, 50, 100)

In [ ]:
for i in [5,10,20,50,100,200]:
  user_user = KNNBasic(k=i, min_k=2, sim_options=sim_options)
  user_user.fit(trainset=train_set)
  pred_user = user_user.test(test_set)
  print('Para {0} vecinos el RMSE es: {1} \n'.format(i, accuracy.rmse( pred_user, verbose = True )))

A medida que aumenta el número de vecinos, el valor del RMSE tiende a disminuir, sin embargo, llega un punto que el sistema de recomendación al tener que considerar tantos vecinos, empiza de nuevo a aumentar el valor del RMSE, en el caso practico se vió este cambio cuando kes 50, porque se incremento el valor del RMSE aunque no es considerable.

##Generando listas de predicciones para los usuarios

Retomemos nuestro modelo inicial y ajustémolo con todos los ratings disponibles

Para generar una lista de recomendación se debe crear un dataset de "test" con las entradas faltantes de la matriz utilidad para que el modelo cree las predicciones (terminar de llenar la matriz de utilidad)



In [ ]:
#Se crea el dataset para modelo 
rating_data=surprise_dataset.build_full_trainset()
# Se crea dataset de "prueba" con las entradas faltantes para generar las predicciones
test=rating_data.build_anti_testset()

# se crea el mismo modelo que el del ejemplo
sim_options = {'name': 'cosine',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=20, min_k=2, sim_options=sim_options)
algo.fit(rating_data)
predictions=algo.test(test)

In [ ]:
rating_data

In [ ]:
#10 primeras predicciones
predictions[0:10]

In [ ]:
#Predicciones para usuario 196
user_predictions=list(filter(lambda x: x[0]==196,predictions))

In [ ]:
#Ordenamos de mayor a menor estimación de relevancia
user_predictions.sort(key=lambda x : x.est, reverse=True)

In [ ]:
#tomamos las 10 primeras predicciones
user_predictions=user_predictions[0:10]

In [ ]:
user_predictions

In [ ]:
#Se convierte a dataframe
labels = ['movie id', 'estimation']
df_predictions = pd.DataFrame.from_records(list(map(lambda x: (x.iid, x.est) , user_predictions)), columns=labels)

In [ ]:
df_predictions

In [ ]:
#Lo unimos con el dataframe de películas
df_predictions.merge(items[['movie title','IMDb URL ']], how='left', right_index=True, left_on='movie id')

## Utilice las siguientes celdas para encontrar la respuesta a las siguientes preguntas

Cree al menos 2 usuarios (al primero asígnele el id 944) y cree para cada usuario ratings nuevos (puede hacerlo haciendo a traves del [dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.append.html), o añadiendo líneas al archivo). Asigne a un usuario preferencias de un segmento popular (mainstream) y a otro preferencias de un nicho.

¿Que tan bien cree que el sistema esta respondiendo a los gustos del usuario?

¿Que tan bien cree que el sistema esta respondiendo al objetivo de buscar items para el usuario dentro de la cola larga?

In [ ]:
ratings['user_id'].max()+1

In [ ]:
ratings.dtypes

In [ ]:
#Peliculas más vistas
calificacion_item[:10]

In [ ]:
#peliculas del nicho 'Documentary'
items[items['Documentary']==1]

Al usuario 944 se le agregan calificaciones de peliculas mainstream, mientras que para el usuario 945 se agregan calificaciones de peliculas de la categoría 'Documentary'

In [ ]:
df2 = pd.DataFrame({"user_id":[944, 944, 944, 945, 945], 
                    "item_id":[50, 1, 121,32,1331],  
                    "rating":[4, 5, 4, 4, 5],
                    "timestamp":[881250949,881250948,881250950,891250949,881251049]}) 

In [ ]:
ratings = ratings.append(df2, ignore_index=True)

In [ ]:
ratings

In [ ]:
#Se crea el dataset a partir del dataframe
surprise_dataset = Dataset.load_from_df( ratings[ [ 'user_id', 'item_id', 'rating' ] ], reader )
#Se crea el dataset para modelo 
rating_data=surprise_dataset.build_full_trainset()
# Se crea dataset de "prueba" con las entradas faltantes para generar las predicciones
test=rating_data.build_anti_testset()

# se crea el mismo modelo que el del ejemplo
sim_options = {'name': 'cosine',
               'user_based': False  # calcule similitud item-item
               }
algo = KNNBasic(k=20, min_k=2, sim_options=sim_options)
algo.fit(rating_data)
predictions=algo.test(test)

In [ ]:
#Predicciones para usuario 944
user_predictions=list(filter(lambda x: x[0]==944,predictions))

In [ ]:
user_predictions.sort(key=lambda x : x.est, reverse=True)

In [ ]:
#Se convierte a dataframe
labels = ['movie id', 'estimation']
df_predictions_944 = pd.DataFrame.from_records(list(map(lambda x: (x.iid, x.est) , user_predictions)), columns=labels)

In [ ]:
#Lo unimos con el dataframe de películas
df_predictions_944.merge(items[['movie title','IMDb URL ']], how='left', right_index=True, left_on='movie id')

Usuario 945

In [ ]:
user_predictions=list(filter(lambda x: x[0]==945,predictions))
user_predictions.sort(key=lambda x : x.est, reverse=True)
labels = ['movie id', 'estimation']
df_predictions_945 = pd.DataFrame.from_records(list(map(lambda x: (x.iid, x.est) , user_predictions)), columns=labels)

In [ ]:
df_predictions_945.merge(items[['movie title','IMDb URL ']], how='left', right_index=True, left_on='movie id')

LensKit

In [ ]:
pip install lenskit

In [ ]:
from lenskit import DataWarning
from lenskit.algorithms import Recommender
from lenskit.algorithms.basic import Fallback
from lenskit.algorithms.bias import Bias
from lenskit import batch
import lenskit.algorithms.item_knn as knn
from lenskit.sharing import persist
from lenskit.util.parallel import run_sp

In [ ]:
ratings.rename(columns={'user_id': 'user'}, inplace = True)
ratings.rename(columns={'item_id': 'item'}, inplace = True)
ratings.head()

In [ ]:
train_set, test_set=  train_test_split(surprise_dataset, test_size=.2)

In [ ]:
algo = knn.ItemItem(30, save_nbrs=500)
algo.fit(ratings)
res = algo.predict_for_user(154,[302])

la libreria Lenskit es una herramienta que permite darle soporte al entrenamiento, validación y evaluación de los algoritmos de recomendación, permitiendo flexibilidad para la investigación y la educación. Lenskit permite realizar filtros colaborativos, item-item y usuario-usuario. Al igual que la libreria surprise permite parametrizar el número máximo y mínimo de vecinos cercanos para calificar cada item. Un parámetro interesante en que tiene el modelo item-item es save_nbrs que hace referencia al número de vecios a guardar por item en el modelo entrenado, permitiendo de uno hasta un valor ilimitado de vecinos.

In [ ]:
res

In [ ]:

user_rmse = res.groupby('item').apply(lambda df: rmse(4.087094, 4))
user_rmse.mean()


Como se puede observar en esta predicción de item-item, se observa que la predicción del modelo es muy similar a la calificación real dada por el usuario, con una calificación predicha de 4.087, mientras que el valor real es 4.